# Get Info Box Details of one film (BvS obviuosly :P)

In [1]:
import requests
from bs4 import BeautifulSoup as bs

In [2]:
# test for one film
bvs_req = requests.get("https://en.wikipedia.org/wiki/Batman_v_Superman:_Dawn_of_Justice").content
bvs_soup = bs(bvs_req)
print(bvs_soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Batman v Superman: Dawn of Justice - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"25886cae-c3de-4863-9fd4-843c462a4c82","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Batman_v_Superman:_Dawn_of_Justice","wgTitle":"Batman v Superman: Dawn of Justice","wgCurRevisionId":1077983590,"wgRevisionId":1077983590,"wgArticleId":2619910,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["All articles lacking reliable references","Articles lacking reliable ref

In [3]:
bvs_infobox = bvs_soup.find("table", attrs={"class": "infobox vevent"}).find("tbody")

In [4]:
bvs_infobox_content = bvs_infobox.find_all("tr")[2:] # remove title and image

In [5]:
bvs_infobox_content

[<tr><th class="infobox-label" scope="row" style="white-space: nowrap; padding-right: 0.65em;">Directed by</th><td class="infobox-data"><a href="/wiki/Zack_Snyder" title="Zack Snyder">Zack Snyder</a></td></tr>,
 <tr><th class="infobox-label" scope="row" style="white-space: nowrap; padding-right: 0.65em;">Written by</th><td class="infobox-data"><div class="plainlist">
 <ul><li><a href="/wiki/Chris_Terrio" title="Chris Terrio">Chris Terrio</a></li>
 <li><a href="/wiki/David_S._Goyer" title="David S. Goyer">David S. Goyer</a></li></ul>
 </div></td></tr>,
 <tr><th class="infobox-label" scope="row" style="white-space: nowrap; padding-right: 0.65em;">Based on</th><td class="infobox-data"><a href="/wiki/Lists_of_DC_Comics_characters" title="Lists of DC Comics characters">Characters</a><br/>from <a href="/wiki/DC_Comics" title="DC Comics">DC Comics</a></td></tr>,
 <tr><th class="infobox-label" scope="row" style="white-space: nowrap; padding-right: 0.65em;">Produced by</th><td class="infobox-da

In [6]:
bvs_dict = {}
bvs_infobox.find_all("tr")[0].text

'Batman v Superman: Dawn of Justice'

In [7]:
bvs_dict["Title"] = bvs_infobox.find_all("tr")[0].text
bvs_dict

{'Title': 'Batman v Superman: Dawn of Justice'}

In [8]:
print(bvs_infobox_content[0])

<tr><th class="infobox-label" scope="row" style="white-space: nowrap; padding-right: 0.65em;">Directed by</th><td class="infobox-data"><a href="/wiki/Zack_Snyder" title="Zack Snyder">Zack Snyder</a></td></tr>


In [9]:
print(bvs_infobox_content[0].find("th").text, bvs_infobox_content[0].find("td").text) # tester for one row

Directed by Zack Snyder


In [10]:
for row in bvs_infobox_content:
  if((row.find("th") != None) and (row.find("td") != None)):
    bvs_dict[str(row.find("th").text)] = str(row.find("td").text).strip().replace("\n", ", ").replace("\xa0", " ")

In [11]:
bvs_dict

{'Based on': 'Charactersfrom DC Comics',
 'Box office': '$873.6 million[6][7]',
 'Budget': '$250–300 million[4][5]',
 'Cinematography': 'Larry Fong',
 'Country': 'United States[2]',
 'Directed by': 'Zack Snyder',
 'Distributed by': 'Warner Bros. Pictures',
 'Edited by': 'David Brenner',
 'Language': 'English',
 'Music by': 'Hans Zimmer, Junkie XL',
 'Produced by': 'Charles Roven, Deborah Snyder',
 'Productioncompanies': 'DC Entertainment[1], RatPac-Dune Entertainment[2], Atlas Entertainment[2], Cruel and Unusual Films[2]',
 'Release dates': 'March 19, 2016 (2016-03-19) (Auditorio Nacional), March 25, 2016 (2016-03-25) (United States)',
 'Running time': '152 minutes[3]',
 'Starring': 'Ben Affleck, Henry Cavill, Amy Adams, Jesse Eisenberg, Diane Lane, Laurence Fishburne, Jeremy Irons, Holly Hunter, Gal Gadot',
 'Title': 'Batman v Superman: Dawn of Justice',
 'Written by': 'Chris Terrio, David S. Goyer'}

# Generalize for list of films

In [12]:
def get_info_for_one_film(url):
  try:
    req = requests.get(url).content
  except requests.exceptions.RequestException as e:  
    return None # error in parsing page
  soup = bs(req)
  infobox_table = soup.find("table", attrs={"class": "infobox vevent"})
  infobox = None
  if(infobox_table != None): # non-english sites have different table classes, edge case
    infobox = infobox_table.find("tbody")
    infobox_content = infobox.find_all("tr")[2:] # remove title and image
    film_dict = {}
    film_dict["Title"] = infobox.find_all("tr")[0].get_text(" ", strip = True)
    for row in infobox_content:
      if((row.find("th") != None) and (row.find("td") != None)):
        if(row.find("li")):
          film_dict[str(row.find("th").get_text(" ", strip = True))] = [li.get_text(" ", strip = True).replace("\xa0", " ") for li in row.find_all("li")] # multi value fields
        else:
          film_dict[str(row.find("th").get_text(" ", strip = True))] = str(row.find("td").get_text(" ", strip = True)).replace("\xa0", " ")
    return film_dict
  else:
    return None

In [13]:
get_info_for_one_film('https://en.wikipedia.org/wiki/Yogi_Bear_(film)') # tester for function

{'Based on': 'The Yogi Bear Show by William Hanna Joseph Barbera',
 'Box office': '$203.5 million [1]',
 'Budget': '$80 million [2] [1]',
 'Cinematography': 'Peter James',
 'Country': 'United States',
 'Directed by': 'Eric Brevig',
 'Distributed by': 'Warner Bros. Pictures',
 'Edited by': 'Kent Beyda',
 'Language': 'English',
 'Music by': 'John Debney',
 'Produced by': ['Donald De Line', 'Karen Rosenfelt'],
 'Production companies': ['De Line Pictures', 'Sunswept Entertainment'],
 'Release dates': ['December 11, 2010 ( 2010-12-11 ) ( Westwood )',
  'December 17, 2010 ( 2010-12-17 ) (United States)'],
 'Running time': '80 minutes [1]',
 'Starring': ['Dan Aykroyd',
  'Justin Timberlake',
  'Anna Faris',
  'Tom Cavanagh',
  'T.J. Miller'],
 'Title': 'Yogi Bear',
 'Written by': ['Brad Copeland', 'Joshua Sternin', 'Jeffrey Ventimilia']}

In [14]:
base_url = 'https://en.wikipedia.org/'
list_page = requests.get('https://en.wikipedia.org/wiki/List_of_Warner_Bros._films_(2010%E2%80%932019)').content # load lisst of WB films in 2010-2020
list_soup = bs(list_page)
list_soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of Warner Bros. films (2010–2019) - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"1899cf1b-0c35-42b8-832b-7ded96209dbb","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_Warner_Bros._films_(2010–2019)","wgTitle":"List of Warner Bros. films (2010–2019)","wgCurRevisionId":1074376744,"wgRevisionId":1074376744,"wgArticleId":65327884,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidat

In [15]:
list_table_body = list_soup.find("table", attrs={"class":"wikitable sortable"}).find("tbody")
print(list_table_body)

<tbody><tr>
<th scope="col" style="width:130px;">Release date
</th>
<th>Title
</th>
<th>Notes
</th></tr>
<tr>
<td style="text-align:right;">January 15, 2010</td>
<td><i><a href="/wiki/The_Book_of_Eli" title="The Book of Eli">The Book of Eli</a></i></td>
<td>US distribution only; produced by <a href="/wiki/Alcon_Entertainment" title="Alcon Entertainment">Alcon Entertainment</a>, <a href="/wiki/Summit_Entertainment" title="Summit Entertainment">Summit Entertainment</a> and <a href="/wiki/Silver_Pictures" title="Silver Pictures">Silver Pictures</a>
</td></tr>
<tr>
<td style="text-align:right;">January 29, 2010</td>
<td><i><a href="/wiki/Edge_of_Darkness_(2010_film)" title="Edge of Darkness (2010 film)">Edge of Darkness</a></i></td>
<td>American, Canadian, Latin American, and Spanish distribution only; co-production with <a href="/wiki/GK_Films" title="GK Films">GK Films</a>, <a href="/wiki/Icon_Productions" title="Icon Productions">Icon Productions</a> and <a class="mw-redirect" href="/wi

In [16]:
list_table_films = list_table_body.find_all("i") # all titles are italicised, fortunately :P
list_films_urls = []
for row in list_table_films:
  anchor_tag = row.find("a")
  if(anchor_tag):
    film_url = anchor_tag.get("href")
    list_films_urls.append(film_url)

In [17]:
len(list_films_urls)

363

In [18]:
films_data = []
for film_url in list_films_urls:
  data = get_info_for_one_film(base_url + film_url)
  print(data)
  if(data != None): # skip for non-english wikipedia urls
    films_data.append(data)

{'Title': 'The Book of Eli', 'Directed by': 'The Hughes Brothers', 'Written by': 'Gary Whitta', 'Produced by': ['Joel Silver', 'Susan Downey', 'Andrew Kosove', 'Broderick Johnson', 'Denzel Washington'], 'Starring': ['Denzel Washington', 'Gary Oldman', 'Mila Kunis', 'Ray Stevenson', 'Jennifer Beals', 'Frances de la Tour', 'Michael Gambon'], 'Cinematography': 'Don Burgess', 'Edited by': 'Cindy Mollo', 'Music by': ['Atticus Ross', 'Leopold Ross', 'Claudia Sarne'], 'Production companies': ['Alcon Entertainment', 'Silver Pictures'], 'Distributed by': ['Warner Bros. Pictures (North America and Turkey)', 'Summit Entertainment [1] [2]', 'Sony Pictures Releasing International [3] [4] (International)'], 'Release date': ['January 15, 2010 ( 2010-01-15 )'], 'Running time': '117 minutes [5]', 'Country': 'United States', 'Language': 'English', 'Budget': '$80 million [6]', 'Box office': '$157.1 million [3]'}
{'Title': 'Edge of Darkness', 'Directed by': 'Martin Campbell', 'Written by': ['William Monah

In [19]:
print(len(films_data))

333


In [20]:
import json
films_json = json.dumps(films_data, indent = 4)

In [21]:
films_json

'[\n    {\n        "Title": "The Book of Eli",\n        "Directed by": "The Hughes Brothers",\n        "Written by": "Gary Whitta",\n        "Produced by": [\n            "Joel Silver",\n            "Susan Downey",\n            "Andrew Kosove",\n            "Broderick Johnson",\n            "Denzel Washington"\n        ],\n        "Starring": [\n            "Denzel Washington",\n            "Gary Oldman",\n            "Mila Kunis",\n            "Ray Stevenson",\n            "Jennifer Beals",\n            "Frances de la Tour",\n            "Michael Gambon"\n        ],\n        "Cinematography": "Don Burgess",\n        "Edited by": "Cindy Mollo",\n        "Music by": [\n            "Atticus Ross",\n            "Leopold Ross",\n            "Claudia Sarne"\n        ],\n        "Production companies": [\n            "Alcon Entertainment",\n            "Silver Pictures"\n        ],\n        "Distributed by": [\n            "Warner Bros. Pictures (North America and Turkey)",\n            "Sum

In [22]:
def write_Data():
  with open("/content/drive/MyDrive/films.json", "w", encoding = 'utf-8') as f: # to write
    json.dump(films_data, f, ensure_ascii = False, indent = 2)  

In [23]:
def load_data():
  with open("/content/drive/MyDrive/films.json", encoding = 'utf-8') as f:
    return json.load(f)

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
write_Data()